# Algorithmic Market Efficiency - Machine Learning Code

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import seaborn as sns

import pickle
import time
from datetime import datetime
import warnings
import os
import sys

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
%config InlineBackend.figure_format = 'retina'

filepath = 'output/'
graphpath = 'graphs/'
tablespath = 'tables/'

print("current directory is : " + os.getcwd())

# Data importation

In [ ]:
df = pickle.load(open('data/df_ml', 'rb'))

dates, dtdates, permnos, vrs, facs, returns, T, N, V, F =  pickle.load(open('data/info', 'rb'))

predictions = pickle.load(open('mloutput/predictions', 'rb'))
scores = pickle.load(open('mloutput/scores', 'rb'))

# Data loading module

In [ ]:
def yearlydataloader(curdates, prevdates):
    
    warnings.filterwarnings('ignore')
    
    print("*** Dataloader ***")

    if len(prevdates) == 0 : sys.exit()

    # Current values
    
    curdf = df[df['date'].isin(curdates)]

    curvalidpermnos = np.unique(np.array(curdf['permno']))

    curvalidpmernomask = ~np.isnan(returns.loc[curdates])

    curdf = curdf.set_index(['date', 'permno'])

    curdf = curdf.stack(dropna=False).unstack(['date','permno'])

    curdf = np.array(curdf).T

    currets, curfacs = curdf[:,0], curdf[:,1:]
    
    # Treatment of nans: drop factors with >50% of nans, replace others by 0

    curnanshares = [np.count_nonzero(np.isnan(curfacs[:,i])) / curfacs.shape[0] for i in range(curfacs.shape[1])]
    curnanshares = np.array(curnanshares)
    
    curvalidfacs_idx = curnanshares<=0.5
    curvalidfacs = facs[curvalidfacs_idx]

    curfacs = curfacs[:, curvalidfacs_idx]
    curfacs = np.nan_to_num(curfacs)

    print("currets shape:", currets.shape)
    print("curfacs shape:", curfacs.shape)

    # Previous values
    
    prevdf = df[df['date'].isin(prevdates)]
    prevdf = prevdf.fillna(0)

    prevdf = prevdf.set_index(['date', 'permno'])
    prevdf = prevdf.stack().unstack(['date','permno'])
    prevdf = np.array(prevdf).T

    prevrets = prevdf[:,0]
    prevfacs = prevdf[:,1:]

    # Treatment of nans: drop currently invalid factors, replace others by 0

    prevfacs = prevfacs[:, curvalidfacs_idx]
    prevfacs = np.nan_to_num(prevfacs)

    # Train-test split for previous factors and returns:

    prevfacs_train, prevfacs_test, prevrets_train, prevrets_test = train_test_split(prevfacs, prevrets, test_size=0.2, random_state=19111997)

    print("prevrets_train shape:", prevrets_train.shape)
    print("prevrets_test shape:", prevrets_test.shape)
    
    print("prevfacs_train shape:", prevfacs_train.shape)
    print("prevfacs_test shape:", prevfacs_test.shape, "\n")

    return curfacs, currets, prevfacs_train, prevfacs_test, prevrets_train, prevrets_test, curvalidpermnos, curvalidfacs, curvalidpmernomask

# Machine learning modules

### OLS

In [ ]:
def ols(prevfacs_train, prevrets_train):
                
    ols_estimator = LinearRegression().fit(prevfacs_train, prevrets_train)
    
    return ols_estimator

### Lasso

In [ ]:
def lasso(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    lasso_parameter = {'alpha':np.logspace(-10,10,100)}
        
    lasso_estimator = GridSearchCV(Lasso(), lasso_parameter, cv=3, return_train_score=True).fit(prevfacs_train, prevrets_train)
    
    paramvalues = lasso_estimator.cv_results_['param_alpha']    
    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split0_train_score'], 'r:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split1_train_score'], 'r:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split2_train_score'], 'r:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['mean_train_score'], 'r', label='train')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split0_test_score'], 'b:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split1_test_score'], 'b:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['split2_test_score'], 'b:')
    plt.semilogx(paramvalues, lasso_estimator.cv_results_['mean_test_score'], 'b', label='test')
    plt.semilogx(paramvalues, np.zeros(len(paramvalues)), 'k--', linewidth=2)
    plt.semilogx(paramvalues, np.ones(len(paramvalues)), 'k--', linewidth=2)
    plt.legend()
    plt.xlabel('Lasso : alpha')
    plt.title("Lasso Regression - 3-Fold Cross-Validation - " + str(curdate))
    plt.show()
    
    return lasso_estimator

### Ridge

In [ ]:
def ridge(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    ridge_parameter = {'alpha':np.logspace(-15,25,100)}
        
    ridge_estimator = GridSearchCV(Ridge(), ridge_parameter, cv=3, return_train_score=True).fit(prevfacs_train, prevrets_train)
            
    paramvalues = ridge_estimator.cv_results_['param_alpha']    
    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split0_train_score'], 'r:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split1_train_score'], 'r:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split2_train_score'], 'r:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['mean_train_score'], 'r', label='train')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split0_test_score'], 'b:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split1_test_score'], 'b:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['split2_test_score'], 'b:')
    plt.semilogx(paramvalues, ridge_estimator.cv_results_['mean_test_score'], 'b', label='test')
    plt.semilogx(paramvalues, np.zeros(len(paramvalues)), 'k--', linewidth=2)
    plt.semilogx(paramvalues, np.ones(len(paramvalues)), 'k--', linewidth=2)
    plt.legend()
    plt.xlabel('Ridge : alpha')
    plt.title("Ridge Regression - 3-Fold Cross-Validation - " + str(curdate))
    plt.show()
    
    return ridge_estimator

### Elastic Net

In [ ]:
def enet(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    alphas = np.logspace(-12,15,10)
    l1_ratios = np.linspace(0,1,10)
    enet_parameter = {'alpha':alphas, 'l1_ratio':l1_ratios}
    
    enet_estimator = GridSearchCV(ElasticNet(), enet_parameter, cv=3, return_train_score=True).fit(prevfacs_train, prevrets_train)
    

    scoresmatrix = np.reshape(enet_estimator.cv_results_['mean_test_score'], (len(alphas),len(l1_ratios)))    
    paramvalues = enet_estimator.cv_results_['param_alpha']

    plt.figure(figsize=(14,10))
    plt.yscale('symlog')
    for i, l1_ratio in enumerate(l1_ratios):
        plt.semilogx(alphas, scoresmatrix[::, i], label='l1_ratio: ' + str(l1_ratios[i] % 0.01))
    plt.semilogx(paramvalues, np.zeros(len(paramvalues)), 'k--', linewidth=2)
    plt.semilogx(paramvalues, np.ones(len(paramvalues)), 'k--', linewidth=2)
    plt.xlabel('Enet: alphas')
    plt.ylabel('Mean test score')
    plt.title("Elastic Net Regression - 3-Fold Cross-Validation - " + str(curdate))
    plt.show()
    
    return enet_estimator

### Principal Components Regression

In [ ]:
def pcr(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
                
    olsfunction = LinearRegression()
    pcafunction = PCA()
    pipe = Pipeline(steps=[('pca', pcafunction), ('ols', olsfunction)])
    
    n_components = np.arange(1,25)
    pcaparameter = {'pca__n_components': n_components}
        
    pcr_estimator = GridSearchCV(pipe, pcaparameter, cv=3, return_train_score=True).fit(prevfacs_train, prevrets_train)

    
    paramvalues = n_components
    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split0_train_score'], 'r:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split1_train_score'], 'r:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split2_train_score'], 'r:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['mean_train_score'], 'r', label='train')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split0_test_score'], 'b:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split1_test_score'], 'b:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['split2_test_score'], 'b:')
    plt.plot(paramvalues, pcr_estimator.cv_results_['mean_test_score'], 'b', label='test')
    plt.plot(paramvalues, np.zeros(len(paramvalues)), 'k--', linewidth=2)
    plt.plot(paramvalues, np.ones(len(paramvalues)), 'k--', linewidth=2)
    plt.legend()
    plt.xlabel('PCR : n_components')
    plt.title("Principal Components Regression - 3-Fold Cross-Validation - " + str(curdate))
    plt.show()

    return pcr_estimator

### Partial Least Squares

In [ ]:
def pls(prevfacs_train, prevrets_train):
        
    # PLSRegression does not allow for 'get_params' so we have to code GridSearchCV manually
    
    trainscores = []
    testscores = []
    
    splits = 3

    for n in np.arange(1,25):

            trainscore, testscore = 0, 0
            
            for train_index, test_index in KFold(n_splits=splits).split(prevfacs_train):

                prevfacs_train_train, prevfacs_train_test = prevfacs_train[train_index], prevfacs_train[test_index]
                prevrets_train_train, prevrets_train_test = prevrets_train[train_index], prevrets_train[test_index]
                
                pls_estimator = PLSRegression(n_components=n, scale=False).fit(prevfacs_train_train, prevrets_train_train)

                trainscore += pls_estimator.score(prevfacs_train_train, prevrets_train_train)
                testscore += pls_estimator.score(prevfacs_train_test, prevrets_train_test)
                
            trainscore = trainscore / splits
            testscore = testscore / splits
            
            trainscores.append(trainscore)
            testscores.append(testscore)
    
    nstar = np.argmin(testscores) + 1
    
    pls_estimator = PLSRegression(n_components=nstar, scale=False).fit(prevfacs_train, prevrets_train)
    
    
    paramvalues = range(1,25)
    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.plot(paramvalues, trainscores, 'r', label='train, 5-cv')
    plt.plot(paramvalues, testscores, 'b', label='test, 5-cv')
    plt.plot(paramvalues, np.zeros(len(paramvalues)), 'k--', linewidth=2)
    plt.plot(paramvalues, np.ones(len(paramvalues)), 'k--', linewidth=2)
    plt.legend()
    plt.xlabel('PLS : n_components')
    plt.title("Partial Least Squares Regression - 3-Fold Cross-Validation - " + str(curdate))
    plt.show()
    
    return pls_estimator

### Regression tree

In [ ]:
def tree(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    tree_estimator = DecisionTreeRegressor().fit(prevfacs_train, prevrets_train)
    
    return tree_estimator

### Boosted tree

In [ ]:
def gbrt(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    gbrt_estimator = GradientBoostingRegressor(warm_start=True).fit(prevfacs_train, prevrets_train)
    
    return gbrt_estimator

### Random forest

In [ ]:
def forest(prevfacs_train, prevrets_train):
    
    warnings.filterwarnings('ignore')
    
    forest_estimator = RandomForestRegressor().fit(prevfacs_train, prevrets_train)
    
    return forest_estimator

### 1-Layer Neural Network

In [ ]:
def nn1(prevfacs_train, prevrets_train):
                
    nn1_parameter = {'hidden_layer_sizes': [(16), (32), (64)],
              'activation': ['logistic'],
              'learning_rate': ['invscaling'],
              'power_t': [0.1, 0.5, 0.9],
              'learning_rate_init': np.logspace(-4,0,5),
              'max_iter': [250],
              'warm_start': [True]}

    nn1_estimator = GridSearchCV(MLPRegressor(), nn1_parameter, cv=2, return_train_score=True, n_jobs=-1).fit(prevfacs_train, prevrets_train)

    testscores = nn1_estimator.cv_results_['mean_test_score']
    trainscores = nn1_estimator.cv_results_['mean_train_score']
    y_pos = np.arange(len(testscores))*3

    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.bar(y_pos, trainscores, color='r', align='center',  label='train, 2-cv')
    plt.bar(y_pos+1, testscores, color='b', align='center',  label='test, 2-cv')
    plt.ylabel('mean_test_score')
    plt.plot(y_pos, np.zeros(len(y_pos)), 'k--', linewidth=2)
    plt.plot(y_pos, np.ones(len(y_pos)), 'k--', linewidth=2)
    plt.legend()
    plt.title("1-Layer Neural Network - 2-Fold Cross-Validation - " + str(curdate))
    plt.show()

    return nn1_estimator

### 2-Layer Neural Network

In [ ]:
def nn2(prevfacs_train, prevrets_train):
                
    nn2_parameter = {'hidden_layer_sizes': [(16,8), (32,16), (64,32)],
              'activation': ['logistic'],
              'learning_rate': ['invscaling'],
              'power_t': [0.1, 0.5, 0.9],
              'learning_rate_init': np.logspace(-4,0,5),
              'max_iter': [250],
              'warm_start': [True]}

    nn2_estimator = GridSearchCV(MLPRegressor(), nn2_parameter, cv=2, return_train_score=True, verbose=1, n_jobs=-1).fit(prevfacs_train, prevrets_train)

    testscores = nn2_estimator.cv_results_['mean_test_score']
    trainscores = nn2_estimator.cv_results_['mean_train_score']
    y_pos = np.arange(len(testscores))*3

    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.bar(y_pos, trainscores, color='r', align='center',  label='train, 2-cv')
    plt.bar(y_pos+1, testscores, color='b', align='center',  label='test, 2-cv')
    plt.ylabel('mean_test_score')
    plt.plot(y_pos, np.zeros(len(y_pos)), 'k--', linewidth=2)
    plt.plot(y_pos, np.ones(len(y_pos)), 'k--', linewidth=2)
    plt.legend()
    plt.title("2-Layer Neural Network - 2-Fold Cross-Validation - " + str(curdate))
    plt.show()


    return nn2_estimator

### 3-Layer Neural Network

In [ ]:
def nn3(prevfacs_train, prevrets_train):
                
    nn3_parameters = {'hidden_layer_sizes': [(16,8,4), (32,16,8), (64,32,16)],
              'activation': ['logistic'],
              'learning_rate': ['invscaling'],
              'power_t': [0.1, 0.5, 0.9],
              'learning_rate_init': np.logspace(-4,0,5),
              'max_iter': [250],
              'warm_start': [True]}

    nn3_estimator = GridSearchCV(MLPRegressor(), nn3_parameters, cv=2, return_train_score=True, n_jobs=-1).fit(prevfacs_train, prevrets_train)

    testscores = nn3_estimator.cv_results_['mean_test_score']
    trainscores = nn3_estimator.cv_results_['mean_train_score']
    
    
    y_pos = np.arange(len(testscores))*3

    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.bar(y_pos, trainscores, color='r', align='center',  label='train, 2-cv')
    plt.bar(y_pos+1, testscores, color='b', align='center',  label='test, 2-cv')
    plt.ylabel('mean_test_score')
    plt.plot(y_pos, np.zeros(len(y_pos)), 'k--', linewidth=2)
    plt.plot(y_pos, np.ones(len(y_pos)), 'k--', linewidth=2)
    plt.legend()
    plt.title("3-Layer Neural Network - 2-Fold Cross-Validation - " + str(curdate))
    plt.show()


    return nn3_estimator

### 5-Layer Neural Network

In [ ]:
def nn5(prevfacs_train, prevrets_train):
                
    nn5_parameter = {'hidden_layer_sizes': [(64,32,16,8,4),(32,16,8,4,2)],
                      'activation': ['relu'],
                      'learning_rate': ['adaptive'],
                      'learning_rate_init': [0.01],
                      'max_iter': [1000],
                      'warm_start': [True]}

    nn5_estimator = GridSearchCV(MLPRegressor(), nn5_parameter, cv=2, return_train_score=True, n_jobs=-1).fit(prevfacs_train, prevrets_train)

    testscores = nn5_estimator.cv_results_['mean_test_score']
    trainscores = nn5_estimator.cv_results_['mean_train_score']
    y_pos = np.arange(len(testscores))*3

    plt.figure(figsize=(10,6))
    plt.yscale('symlog')
    plt.bar(y_pos, trainscores, color='r', align='center',  label='train, 2-cv')
    plt.bar(y_pos+1, testscores, color='b', align='center',  label='test, 2-cv')
    plt.ylabel('mean_test_score')
    plt.plot(y_pos, np.zeros(len(y_pos)), 'k--', linewidth=2)
    plt.plot(y_pos, np.ones(len(y_pos)), 'k--', linewidth=2)
    plt.legend()
    plt.title("5-Layer Neural Network - 2-Fold Cross-Validation - " + str(curdate))
    plt.show()


    return nn5_estimator

### 10-Layer Neural Network

In [ ]:
def nn10(prevfacs_train, prevrets_train):
                
    nn10_estimator = MLPRegressor(hidden_layer_sizes=(100, 90, 80, 70, 60, 50, 40, 30, 20, 10),
                                    activation='relu',
                                    learning_rate='adaptive',
                                    learning_rate_init=0.01,
                                    max_iter=1000,
                                    warm_start=True).fit(prevfacs_train, prevrets_train)

    return nn10_estimator

# Core Loop

In [ ]:
methods = ['ols', 'lasso', 'ridge', 'enet', 'pcr', 'pls', 'tree', 'gbrt', 'forest', 'nn1', 'nn2', 'nn3', 'nn5', 'nn10']
M = len(methods)

splits = ['train', 'test', 'oos']
scoretypes = ['R2', 'spearman']

if os.path.exists('mloutput/scores'):
    scores = pickle.load(open('mloutput/scores', 'rb'))
else:
    scoreindex = pd.MultiIndex.from_product([dates, methods, splits, scoretypes], names=['date', 'method', 'split', 'scoretype'])
    scores = pd.Series(np.nan, index=scoreindex)
    pickle.dump(scores, open('mloutput/scores', 'wb'))
    
if os.path.exists('mloutput/predictions'):
    predictions = pickle.load(open('mloutput/predictions', 'rb'))
else:
    predictions = returns.copy()
    # We set unfilled predictions to exactly pi: this way we preserve the nan structure of the returns matrix,
    # but can easily check whether in the end some values were not overwritten by actual predictitons.
    predictions[~np.isnan(predictions)] = np.pi
    predictions = dict(zip(methods,[predictions] * M))
    pickle.dump(predictions, open('mloutput/predictions', 'wb'))

In [ ]:
runmethods = [ols]
runmethodnames = [runmethod.__name__ for runmethod in runmethods]

delta = 12*10
override = False

for t in range(12,T,12):
    
    print("*** New Loop ***")
    
    curdate = dates[t]  
    curdates = np.array(dates[t:t+12])
    prevdates = np.array(dates[max(t-delta,0):t])
    print("Out-of-sample dates:", curdates)
    print("Train/test dates:", prevdates)

    if np.isnan(scores.loc[(curdate,runmethodnames,'train','R2')]).any()==False and override==False:
            print('All done!  Date : ', curdate, '\n')
            continue
    
    curfacs, currets, prevfacs_train, prevfacs_test, prevrets_train, prevrets_test, curvalidpermnos, curvalidfacs, curvalidpmernomask = yearlydataloader(curdates, prevdates)
    
    for method in runmethods:
        
        curmethod = method.__name__

        print("*** Current method: ", curmethod, " ***")
        
        if np.isnan(scores.loc[(curdate,curmethod,'train','R2')]).any()==False and override==False:
            print('Already done!  Date : ', curdate, '   Method : ', curmethod, '\n')
            continue
        
        estimator = method(prevfacs_train, prevrets_train)
        

        prediction_train = estimator.predict(prevfacs_train)
        prediction_test = estimator.predict(prevfacs_test)
        prediction = estimator.predict(curfacs)
        
        predictionscopy = predictions[curmethod].copy()

        predictions_unmasked = predictionscopy.loc[curdates].to_numpy()
        np.place(predictions_unmasked, curvalidpmernomask.values, prediction.flatten())

        predictionscopy.loc[curdates] = predictions_unmasked
        predictions[curmethod] = predictionscopy

        # This part is very confusing, because .loc interacts weirdly with dictionnaries and overwrites them...
        
        scores.loc[(curdate,curmethod,'train','R2')] = r2_score(prevrets_train, prediction_train)
        scores.loc[(curdate,curmethod,'train','spearman')] = sp.stats.spearmanr(prevrets_train, prediction_train)[0]

        scores.loc[(curdate,curmethod,'test','R2')] = r2_score(prevrets_test, prediction_test)
        scores.loc[(curdate,curmethod,'test','spearman')] = sp.stats.spearmanr(prevrets_test, prediction_test)[0]

        scores.loc[(curdate,curmethod,'oos','R2')] = r2_score(currets, prediction)
        scores.loc[(curdate,curmethod,'oos','spearman')] = sp.stats.spearmanr(currets, prediction)[0]

        print("Train R2:", scores.loc[(curdate,curmethod,'train','R2')])
        print("Train spearman:", scores.loc[(curdate,curmethod,'train','spearman')])
        
        print("Test R2:", scores.loc[(curdate,curmethod,'test','R2')])
        print("Test spearman:", scores.loc[(curdate,curmethod,'test','spearman')])

        print("Out-of-sample R2:", scores.loc[(curdate,curmethod,'oos','R2')])
        print("Out-of-sample Spearman:", scores.loc[(curdate,curmethod,'oos','spearman')])

        print('Great sucess!  Date : ', curdate, ' Method : ', curmethod, '\n')

        pickle.dump(predictions, open('mloutput/predictions', 'wb'))
        pickle.dump(scores, open('mloutput/scores', 'wb'))

# Model Analysis

In [ ]:
def dataloader2016(curdates, prevdates):
            
    curdf = df[df['date'].isin(curdates)]
    curdf = curdf.set_index(['date', 'permno'])
    
    curdf = curdf.stack(dropna=False).unstack(['date','permno'])
    curdf = np.array(curdf).T

    currets = curdf[:,0]
    curfacs = curdf[:,1:]

    # Previous values
    
    prevdf = df[df['date'].isin(prevdates)]
    prevdf = prevdf.fillna(0)

    prevdf = prevdf.set_index(['date', 'permno'])
    prevdf = prevdf.stack().unstack(['date','permno'])
    prevdf = np.array(prevdf).T

    prevrets = prevdf[:,0]
    prevfacs = prevdf[:,1:]

    # Train-test split for previous factors and returns:

    prevfacs_train, prevfacs_test, prevrets_train, prevrets_test = train_test_split(prevfacs, prevrets, test_size=0.2, random_state=19111997)
    
    # Converting Nans to zeros
    
    prevfacs_train = np.nan_to_num(prevfacs_train)
    prevfacs_test = np.nan_to_num(prevfacs_test)
    curfacs = np.nan_to_num(curfacs)

    return curfacs, currets, prevfacs_train, prevfacs_test, prevrets_train, prevrets_test

t = len(dates) - 12
curdate = dates[t]  
curdates = np.array(dates[t:t+12])
prevdates = np.array(dates[max(t-delta,0):t])

curfacs, currets, prevfacs_train, prevfacs_test, prevrets_train, prevrets_test = dataloader2016(curdates, prevdates)

In [ ]:
testmethods = [ols, lasso, ridge, enet, pcr, pls, tree, gbrt, forest, nn1, nn2, nn3, nn5, nn10]
testmethodnames = [testmethod.__name__ for testmethod in testmethods]
testmethodlabels = ['OLS', 'Lasso', 'Ridge', 'Enet', 'PCR', 'PLS', 'Tree', 'GBRT', 'Forest', 'NN1', 'NN2', 'NN3', 'NN5', 'NN10']

if os.path.exists('mloutput/estimators2016/full/estimatorsdict'):
    estimatorsdict = pickle.load(open('mloutput/estimators2016/full/estimatorsdict', 'rb'))
else: estimatorsdict = dict()

for method in testmethods:
    
    methodname = method.__name__

    print("*** Current method: ", methodname, " ***")
    
    if methodname in estimatorsdict:
        print("Already done!")
        continue

    estimator = method(prevfacs_train, prevrets_train)

    estimatorsdict[methodname] = estimator
    
    pickle.dump(estimatorsdict, open('mloutput/estimators2016/full/estimatorsdict', 'wb'))
    
    print("All done!")

In [ ]:
if os.path.exists('mloutput/estimators2016/full/partspearmans'):
    
    basescores_train, basescores_test, basescores_oos = pickle.load(open('mloutput/estimators2016/full/basescores', 'rb'))
    partscores_train, partscores_test, partscores_oos = pickle.load(open('mloutput/estimators2016/full/partscores', 'rb'))
    
    basespearmans_train, basespearmans_test, basespearmans_oos = pickle.load(open('mloutput/estimators2016/full/basespearmans', 'rb'))
    partspearmans_train, partspearmans_test, partspearmans_oos = pickle.load(open('mloutput/estimators2016/full/partspearmans', 'rb'))
    
else:
    
    basescores_train = np.full(len(testmethods), np.nan)
    partscores_train = np.full([F, len(testmethods)], np.nan)
    basescores_test = np.full(len(testmethods), np.nan)
    partscores_test = np.full([F, len(testmethods)], np.nan)
    basescores_oos = np.full(len(testmethods), np.nan)
    partscores_oos = np.full([F, len(testmethods)], np.nan)
    
    basespearmans_train = np.full(len(testmethods), np.nan)
    partspearmans_train = np.full([F, len(testmethods)], np.nan)
    basespearmans_test = np.full(len(testmethods), np.nan)
    partspearmans_test = np.full([F, len(testmethods)], np.nan)
    basespearmans_oos = np.full(len(testmethods), np.nan)
    partspearmans_oos = np.full([F, len(testmethods)], np.nan)

for i, method in enumerate(testmethods):
    
    continue
            
    methodname = method.__name__
    estimator = estimatorsdict[methodname]
    
    prediction_train = estimator.predict(prevfacs_train)
    prediction_test = estimator.predict(prevfacs_test)
    prediction_oos = estimator.predict(curfacs)
    
    basescores_train[i] = r2_score(prevrets_train, prediction_train)
    basescores_test[i] = r2_score(prevrets_test, prediction_test)
    basescores_oos[i] = r2_score(currets, prediction_oos)
    
    basespearmans_train[i] = sp.stats.spearmanr(prevrets_train, prediction_train)[0]
    basespearmans_test[i] = sp.stats.spearmanr(prevrets_test, prediction_test)[0]
    basespearmans_oos[i] = sp.stats.spearmanr(currets, prediction_oos)[0]
        
    for j in range(F):
        
        if (j%25)==0: print('Method:', methodname, 'Factor:', j+1, 'out of', F)
                                      
        prevfacs_train_factorsettozero = prevfacs_train.copy()
        prevfacs_train_factorsettozero[:,j] = 0
        
        prevfacs_test_factorsettozero = prevfacs_test.copy()
        prevfacs_test_factorsettozero[:,j] = 0
        
        curfacs_factorsettozero = curfacs.copy()
        curfacs_factorsettozero[:,j] = 0
        
        prediction_train_factorsettozero = estimator.predict(prevfacs_train_factorsettozero)           
        partscores_train[j,i] = r2_score(prevrets_train, prediction_train_factorsettozero)
        partspearmans_train[j,i] = sp.stats.spearmanr(prevrets_train, prediction_train_factorsettozero)[0]

        prediction_test_factorsettozero = estimator.predict(prevfacs_test_factorsettozero)           
        partscores_test[j,i] = r2_score(prevrets_test, prediction_test_factorsettozero)
        partspearmans_test[j,i] = sp.stats.spearmanr(prevrets_test, prediction_test_factorsettozero)[0]
        
        prediction_oos_factorsettozero = estimator.predict(curfacs_factorsettozero)           
        partscores_oos[j,i] = r2_score(currets, prediction_oos_factorsettozero)
        partspearmans_oos[j,i] = sp.stats.spearmanr(currets, prediction_oos_factorsettozero)[0]
        
    pickle.dump([basescores_train, basescores_test, basescores_oos], open('mloutput/estimators2016/full/basescores', 'wb'))
    pickle.dump([partscores_train, partscores_test, partscores_oos], open('mloutput/estimators2016/full/partscores', 'wb'))
    
    pickle.dump([basespearmans_train, basespearmans_test, basespearmans_oos], open('mloutput/estimators2016/full/basespearmans', 'wb'))
    pickle.dump([partspearmans_train, partspearmans_test, partspearmans_oos], open('mloutput/estimators2016/full/partspearmans', 'wb'))

In [ ]:
def factorimportancetable(split, scoretype):

    m = len(testmethodlabels)
    
    if scoretype=='R2':
        if split=='train': partscores, basescores = partscores_train, basescores_train
        elif split=='test': partscores, basescores = partscores_test, basescores_test
        elif split=='oos': partscores, basescores = partscores_oos, basescores_oos
    elif scoretype=='spearman':
        if split=='train': partscores, basescores = partspearmans_train, basespearmans_train
        elif split=='test': partscores, basescores = partspearmans_test, basespearmans_test
        elif split=='oos': partscores, basescores = partspearmans_oos, basespearmans_oos


    matrix = - (partscores - basescores) / basescores
    matrix[matrix<0.00001] = 0.00001
    matrix[matrix>1] = 1

    matrix = matrix / np.nansum(matrix, axis=0)

    sumbyfac = np.nansum(matrix, axis=1)
    sortidxs = sumbyfac.argsort()
    
    matrix = matrix[sortidxs[::-1],:]
    facs_sorted = facs[sortidxs[::-1]]


    fig, ax = plt.subplots(figsize = (11,14))
    im = ax.imshow(matrix, aspect= "auto", cmap="Purples", norm=LogNorm(vmin=0.00001, vmax=1))
    
    ax.set_xticks(np.arange(len(testmethodlabels)))
    ax.set_yticks(np.arange(len(facs)))

    ax.set_xticklabels(testmethodlabels, size=12)
    ax.set_yticklabels(facs_sorted, size=8)

    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
    plt.setp(ax.get_xticklabels(), rotation=-45, ha="right", rotation_mode="anchor")

    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(matrix.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(matrix.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=1.5)
    ax.tick_params(which="minor", bottom=False, left=False)
    ax.tick_params(which="major", top=False, left=False)
    
    fig.tight_layout()
    plt.savefig(graphpath+'factorimportancetable_'+split+'_'+scoretype+'.pdf')
    plt.show()
    
factorimportancetable('train', 'R2')
factorimportancetable('train', 'spearman')

factorimportancetable('test', 'R2')
factorimportancetable('test', 'spearman')

factorimportancetable('oos', 'R2')
factorimportancetable('oos', 'spearman')

In [ ]:
### Transposed version for the slides:

def factorimportancetable(split, scoretype):

    m = len(testmethodlabels)
    
    if scoretype=='R2':
        if split=='train': partscores, basescores = partscores_train, basescores_train
        elif split=='test': partscores, basescores = partscores_test, basescores_test
        elif split=='oos': partscores, basescores = partscores_oos, basescores_oos
    elif scoretype=='spearman':
        if split=='train': partscores, basescores = partspearmans_train, basespearmans_train
        elif split=='test': partscores, basescores = partspearmans_test, basespearmans_test
        elif split=='oos': partscores, basescores = partspearmans_oos, basespearmans_oos


    matrix = - (partscores - basescores) / basescores
    matrix[matrix<0.00001] = 0.00001
    matrix[matrix>1] = 1

    matrix = matrix / np.nansum(matrix, axis=0)

    sumbyfac = np.nansum(matrix, axis=1)
    sortidxs = sumbyfac.argsort()
    
    matrix = matrix[sortidxs[::-1],:]
    facs_sorted = facs[sortidxs[::-1]]


    fig, ax = plt.subplots(figsize = (14,7))
    im = ax.imshow(matrix.T, aspect= "auto", cmap="Purples", norm=LogNorm(vmin=0.00001, vmax=1))
    
    ax.set_yticks(np.arange(len(testmethodlabels)))
    ax.set_xticks(np.arange(len(facs)))

    ax.set_yticklabels(testmethodlabels, size=12)
    ax.set_xticklabels(facs_sorted, size=8)

    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
    plt.setp(ax.get_xticklabels(), rotation=90, ha="left", rotation_mode="anchor")

    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(matrix.T.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(matrix.T.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=1.5)
    ax.tick_params(which="minor", bottom=False, left=False)
    ax.tick_params(which="major", top=False, left=False)
    
    fig.tight_layout()
    plt.savefig(graphpath+'factorimportancetable_transpose_'+split+'_'+scoretype+'.pdf')
    plt.show()
    
factorimportancetable('train', 'R2')
factorimportancetable('oos', 'R2')

In [ ]:
palettecycle = sns.color_palette('muted', 4)

keyfacs = ['beta', 'mvel1', 'bm', 'mom6m']    
keyfaclabels = ['Beta', 'Log Market Equity ', 'Book-to-market', '6-Month Momentum']    
keyfaclabels2 = ['Beta', 'Log Market\n Equity ', 'Book-to-market', '6-Month \n Momentum']    

def predictoranalysisplot(estimator, estimatorname, estimatorlabel, titlestr):
    
    pts = 100

    fig, ax = plt.subplots(len(keyfacs), len(keyfacs), figsize=(14,14*0.65))
    
    for axis in ax.flatten():
        axis.spines['right'].set_visible(False)
        axis.spines['top'].set_visible(False)

    for (y, facy) in enumerate(keyfacs):

        facyidx = list(facs).index(facy)
                
        for (x, facx) in enumerate(keyfacs):

            facxidx = list(facs).index(facx)
            
            xm2 = np.zeros((pts, F))
            xm1 = np.zeros((pts, F))
            x0 = np.zeros((pts, F))
            xp1 = np.zeros((pts, F))
            xp2 = np.zeros((pts, F))

            stdspace = np.linspace(-2.1, 2.1, pts)
            
            xm2[:,facxidx] = stdspace.copy()
            xm1[:,facxidx] = stdspace.copy()
            x0[:,facxidx] = stdspace.copy()
            xp1[:,facxidx] = stdspace.copy()
            xp2[:,facxidx] = stdspace.copy()
            
            xm2[:,facyidx] = -2
            xm1[:,facyidx] = -1
            x0[:,facyidx] = 0
            xp1[:,facyidx] = 1
            xp2[:,facyidx] = 2
            
            predictedreturnsm2 = estimator.predict(xm2)
            predictedreturnsm1 = estimator.predict(xm1)
            predictedreturns0 = estimator.predict(x0)
            predictedreturnsp1 = estimator.predict(xp1)
            predictedreturnsp2 = estimator.predict(xp2)
            
            if x==y:
                
                x0[:,facxidx] = stdspace.copy()
                predictedreturns0 = estimator.predict(x0)
                
                predictedreturnsm2 = predictedreturns0
                predictedreturnsm1 = predictedreturns0
                predictedreturnsp1 = predictedreturns0
                predictedreturnsp2 = predictedreturns0
            
            
            colors = sns.light_palette(palettecycle[y], reverse=True)
            
            ax[y, x].plot(stdspace, predictedreturnsm2, c=colors[0], label='- 2 std')
            ax[y, x].plot(stdspace, predictedreturnsm1, c=colors[1], label='- 1 std')
            ax[y, x].plot(stdspace, predictedreturns0, c=colors[2], label='   avg')
            ax[y, x].plot(stdspace, predictedreturnsp1, c=colors[3], label='+1 std')
            ax[y, x].plot(stdspace, predictedreturnsp2, c=colors[4], label='+2 std')
            ax[y, x].plot(stdspace, predictedreturns0, c=colors[2])

            ax[y, x].set_xlabel(facs[facxidx] + ' (stds)')
            ax[y, x].set_ylabel('return')
            ax[y, x].tick_params(axis='both', which='major', labelsize=8)
            ax[y, x].xaxis.set_major_locator(plt.MaxNLocator(3))
            ax[y, x].yaxis.set_major_locator(plt.MaxNLocator(3))
            ax[y, x].yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=1, symbol=None))
            
            if y==0:
                ax[y, x].set_title(keyfaclabels[x], y=1.2, fontsize=11)
            
            if x==0:
                l = ax[y, x].legend(frameon=False, title=keyfaclabels2[y], bbox_to_anchor=(-0.3,0.5), loc="center right")
                plt.setp(l.get_title(), multialignment='center', fontsize=11)
                l._legend_box.sep = 10
    
    plt.suptitle(titlestr, y=1.03, fontsize=13)
    fig.tight_layout(pad=1.0)
    plt.savefig(graphpath+'predictoranalysisplot_'+estimatorname+'.pdf', bbox_inches='tight')
    plt.show()


predictoranalysisplot(estimatorsdict['ols'], 'ols', 'OLS', 'OLS Estimator Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['pcr'], 'pcr', 'PCR', 'PCR Estimator Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['tree'], 'tree', 'Tree', 'Regression Tree Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['gbrt'], 'gbrt', 'GBRT', 'Gradient-Boosted Regression Tree Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['forest'], 'forest', 'Forest', 'Random Forest Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['nn1'], 'nn1', 'NN1', '1-Layer Neural Network Estimator Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['nn2'], 'nn2', 'NN2', '2-Layer Neural Network Trained over 2005-2015')
predictoranalysisplot(estimatorsdict['nn3'], 'nn3', 'NN3', '3-Layer Neural Network Trained over 2005-2015')


## Performance of ML algorithms on (random) subsets of factors 

In [ ]:
np.random.seed(19111997)

facidxlists_10p = dict()
for i in range(10): facidxlists_10p[i] = np.random.randint(0, F, int(F*(10/100)))

facidxlists_25p = dict()
for i in range(10): facidxlists_25p[i] = np.random.randint(0, F, int(F*(25/100)))

facidxlists_50p = dict()
for i in range(10): facidxlists_50p[i] = np.random.randint(0, F, int(F*(50/100)))

facidxlists_75p = dict()
for i in range(10): facidxlists_75p[i] = np.random.randint(0, F, int(F*(75/100)))

facidxlists_90p = dict()
for i in range(10): facidxlists_90p[i] = np.random.randint(0, F, int(F*(90/100)))


subsettestmethods = [ols, pcr, tree, forest, nn1, nn2, nn3]
subsettestmethodnames = ['ols', 'pcr', 'tree', 'forest', 'nn1', 'nn2', 'nn3']
subsettestmethodlabels = ['OLS', 'PCR', 'Tree', 'Forest', 'NN1', 'NN2', 'NN3']
subsettestmethodcolors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:len(subsettestmethods)]

samples = ['train', 'test', 'oos']
scoretypes = ['R2', 'spearman']
subsets = ['10%', '25%', '50%', '75%', '90%']
splits = np.arange(10)

if not os.path.exists('mloutput/estimators2016/subsetscores'):
    subsetscoreindex = pd.MultiIndex.from_product([subsettestmethodnames, samples, scoretypes, subsets, splits], names=['method', 'sample', 'scoretype', 'subset', 'split'])
    subsetscores = pd.Series(np.nan, index=subsetscoreindex)
    print('Redefining!')
else:
    subsetscores = pickle.load(open('mloutput/estimators2016/subsetscores', 'rb'))
    print('Using loaded scores!')

In [ ]:
for subset in subsets:

    for (i, splitnumber) in enumerate(range(10)):
        
        if subset == '10%': splitfacidxs = facidxlists_10p[splitnumber]
        elif subset == '25%': splitfacidxs = facidxlists_25p[splitnumber]
        elif subset == '50%': splitfacidxs = facidxlists_50p[splitnumber]
        elif subset == '75%': splitfacidxs = facidxlists_75p[splitnumber]
        elif subset == '90%': splitfacidxs = facidxlists_90p[splitnumber]
        elif subset == '100%':
            splitfacidxs = np.arange(len(facs))
            if i!=0: continue

        for method in subsettestmethods:
            
            methodname = method.__name__
            
            if (methodname == 'pcr') & (subset == '10%') : continue
            if (methodname == 'pls') & (subset == '10%') : continue
            
            if os.path.exists('mloutput/estimators2016/' + subset + '/estimator_split' + str(splitnumber) + '_' + methodname):
                estimator = pickle.load(open('mloutput/estimators2016/' + subset + '/estimator_split' + str(splitnumber) + '_' + methodname, 'rb'))
                print("Current subset:", subset, "Current split :", splitnumber, "Current method: ", methodname, '-> Already estimated!')
                continue
            else:
                print("Current subset:", subset, "Current split :", splitnumber, "Current method: ", methodname, '-> To do!')
                estimator = method(prevfacs_train[:,splitfacidxs], prevrets_train)

            prediction_train = estimator.predict(prevfacs_train[:,splitfacidxs])
            subsetscores.loc[(methodname,'train','R2', subset, splitnumber)] = r2_score(prevrets_train, prediction_train)
            subsetscores.loc[(methodname,'train','spearman', subset, splitnumber)] = sp.stats.spearmanr(prevrets_train, prediction_train)[0]

            prediction_test = estimator.predict(prevfacs_test[:,splitfacidxs])
            subsetscores.loc[(methodname,'test','R2', subset, splitnumber)] = r2_score(prevrets_test, prediction_test)
            subsetscores.loc[(methodname,'test','spearman', subset, splitnumber)] = sp.stats.spearmanr(prevrets_test, prediction_test)[0]

            prediction_oos = estimator.predict(curfacs[:,splitfacidxs])
            subsetscores.loc[(methodname,'oos','R2', subset, splitnumber)] = r2_score(currets, prediction_oos)
            subsetscores.loc[(methodname,'oos','spearman', subset, splitnumber)] = sp.stats.spearmanr(currets, prediction_oos)[0]

            pickle.dump(estimator, open('mloutput/estimators2016/' + subset + '/estimator_split' + str(splitnumber) + '_' + methodname, 'wb'))
            pickle.dump(subsetscores, open('mloutput/estimators2016/subsetscores', 'wb'))
            pickle.dump(subsetscores, open('mloutput/estimators2016/subsetscores2', 'wb'))

print("All done!")

In [ ]:
subsetscores_splitavgs = subsetscores.groupby(['method', 'sample', 'scoretype', 'subset']).mean()

fig, ax = plt.subplots(3, 2, figsize=(0.75*20, 20))
ax = ax.flatten()

for axis in ax:
    axis.spines['right'].set_visible(False)
    axis.spines['top'].set_visible(False)
        
    for tk in axis.get_yticklabels():
        tk.set_visible(True)
    
def scores_scatter(ax, split, scoretype, titlestr):
    
    s = subsetscores_splitavgs.loc[:,split,scoretype]

    for (i, methodname) in enumerate(subsettestmethodnames):

        ax.plot(subsets, s.loc[methodname].values, subsettestmethodcolors[i], marker='o', linestyle='', markersize=6, label=subsettestmethodlabels[i])
        ax.plot('100%', scores.loc[20160129, methodname, split, scoretype], subsettestmethodcolors[i], marker='o', markersize=6)
  
    ols_labels = np.concatenate((subsets, ['100%']))
    ols_scores = np.concatenate((s.loc['ols'].values, [scores.loc[20160129, 'ols', split, scoretype]]))
    ax.plot(ols_labels, ols_scores, 'C0--')

    ax.set_title(titlestr)
    ax.set_xlabel('Percentage of sampled factors')
    ax.set_ylabel('Average score across 10 splits')    
    ax.legend(frameon=False)


scores_scatter(ax[0], 'train', 'R2', 'Train R2')
scores_scatter(ax[1], 'train', 'spearman', 'Train Spearman')
scores_scatter(ax[2], 'test', 'R2', 'Test R2')
scores_scatter(ax[3], 'test', 'spearman', 'Test Spearman')
scores_scatter(ax[4], 'oos', 'R2', 'Out-of-sample R2')
scores_scatter(ax[5], 'oos', 'spearman', 'Out-of-sample Spearman')

ax[0].set_ylim([-0.02, 1.02])
ax[1].set_ylim([-0.02, 1.02])

ax[2].set_ylim([-0.02, 0.75])
ax[3].set_ylim([-0.02, 0.75])

ax[4].set_ylim([-0.02, 0.75])
ax[5].set_ylim([-0.02, 0.75])

plt.savefig(graphpath+'subsamplescores.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Transposed version for the slides

subsetscores_splitavgs = subsetscores.groupby(['method', 'sample', 'scoretype', 'subset']).mean()

fig, ax = plt.subplots(2, 3, figsize=(16, 10))
ax = ax.flatten()

for axis in ax:
    axis.spines['right'].set_visible(False)
    axis.spines['top'].set_visible(False)
        
    for tk in axis.get_yticklabels():
        tk.set_visible(True)
    
def scores_scatter(ax, split, scoretype, titlestr):
    
    s = subsetscores_splitavgs.loc[:,split,scoretype]

    for (i, methodname) in enumerate(subsettestmethodnames):

        ax.plot(subsets, s.loc[methodname].values, subsettestmethodcolors[i], marker='o', linestyle='', markersize=6, label=subsettestmethodlabels[i])
        ax.plot('100%', scores.loc[20160129, methodname, split, scoretype], subsettestmethodcolors[i], marker='o', markersize=6)
  
    ols_labels = np.concatenate((subsets, ['100%']))
    ols_scores = np.concatenate((s.loc['ols'].values, [scores.loc[20160129, 'ols', split, scoretype]]))
    ax.plot(ols_labels, ols_scores, 'C0--')

    ax.set_title(titlestr)
    ax.set_xlabel('Percentage of sampled factors')
    ax.set_ylabel('Average score across 10 splits')
    
    if not (titlestr=='Train Spearman' or titlestr=='Train R2'): ax.legend(frameon=False)


scores_scatter(ax[0], 'train', 'R2', 'Train R2')
scores_scatter(ax[1], 'test', 'R2', 'Test R2')
scores_scatter(ax[2], 'oos', 'R2', 'Out-of-sample R2')
scores_scatter(ax[3], 'train', 'spearman', 'Train Spearman')
scores_scatter(ax[4], 'test', 'spearman', 'Test Spearman')
scores_scatter(ax[5], 'oos', 'spearman', 'Out-of-sample Spearman')

ax[0].set_ylim([-0.02, 1.02])
ax[3].set_ylim([-0.02, 1.02])

ax[1].set_ylim([-0.02, 0.75])
ax[4].set_ylim([-0.02, 0.75])

ax[2].set_ylim([-0.02, 0.75])
ax[5].set_ylim([-0.02, 0.75])

plt.tight_layout(pad=3.0)
plt.savefig(graphpath+'subsamplescores_transposed.pdf', bbox_inches='tight')
plt.show()